In [1]:
!pip install imblearn


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


<a href="https://loan-eligibility-prediction-feature-engineering.streamlit.app/">App Link<a/>

In [196]:
import warnings 
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
import imblearn
from sklearn.impute import KNNImputer , SimpleImputer

#from sklearn.metrics import confusion_matrix,classification_report
#import category_encoders 
#from category_encoders import BinaryEncoder,HashingEncoder
from sklearn.preprocessing import OneHotEncoder
from imblearn.pipeline import Pipeline , make_pipeline
from imblearn.over_sampling import SMOTE ,SMOTEN ,SMOTENC, SVMSMOTE ,BorderlineSMOTE 


from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.linear_model import LogisticRegression , Lasso ,LassoCV , RidgeClassifier , RidgeClassifierCV ,Ridge ,RidgeCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion
from sklearn.feature_selection import SequentialFeatureSelector,RFE

from sklearn.model_selection import cross_validate , GridSearchCV
#from sklearn.metrics import RocCurveDisplay

import joblib as jb

In [111]:
ml = pd.read_csv('loans_cleaned.csv')

In [112]:
ml

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0.0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1
1,LP001003,Male,Yes,1.0,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0
2,LP001005,Male,Yes,0.0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1
3,LP001006,Male,Yes,0.0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1
4,LP001008,Male,No,0.0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0.0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,1
610,LP002979,Male,Yes,3.0,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,1
611,LP002983,Male,Yes,1.0,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,1
612,LP002984,Male,Yes,2.0,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,1


In [113]:
ml.drop('Loan_ID',axis=1,inplace=True)

In [114]:
ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             601 non-null    object 
 1   Married            611 non-null    object 
 2   Dependents         599 non-null    float64
 3   Education          614 non-null    object 
 4   Self_Employed      582 non-null    object 
 5   ApplicantIncome    614 non-null    int64  
 6   CoapplicantIncome  614 non-null    float64
 7   LoanAmount         592 non-null    float64
 8   Loan_Amount_Term   600 non-null    float64
 9   Credit_History     564 non-null    float64
 10  Property_Area      614 non-null    object 
 11  Loan_Status        614 non-null    int64  
dtypes: float64(5), int64(2), object(5)
memory usage: 57.7+ KB


In [115]:
ml.duplicated().sum()

0

In [116]:
ml.nunique()

Gender                 2
Married                2
Dependents             4
Education              2
Self_Employed          2
ApplicantIncome      505
CoapplicantIncome    287
LoanAmount           203
Loan_Amount_Term      10
Credit_History         2
Property_Area          3
Loan_Status            2
dtype: int64

In [117]:
ml.isna().sum()

Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [118]:
px.imshow(ml.corr(numeric_only=True),text_auto=True , width=1000,height=1200)

# Feature Engineering

In [119]:
ml['min_installment'] = ml['LoanAmount']*1000 *(1-.2) / ml['Loan_Amount_Term']  # supposed 20% pre for loan 

In [120]:
ml['ratio_of_income'] = ml['min_installment'] / ml['ApplicantIncome']*100

In [121]:
ml['ratio_of_income'].describe()

count    578.000000
mean      10.120997
std       20.051338
min        0.670253
25%        5.484486
50%        7.074611
75%       10.023864
max      373.931624
Name: ratio_of_income, dtype: float64

In [122]:
#df['ratio_of_all_incomes'] = df['min_installment'] / (df['ApplicantIncome'] + df['CoapplicantIncome'])*100

In [123]:
#df['diff'] = df['ApplicantIncome'] -  df['min_installment']

In [124]:
#the rest of whole incomes after minimum installment
#ml['diff_all'] = ml['ApplicantIncome'] + ml['CoapplicantIncome'] -  ml['min_installment']   

In [125]:
#ml['diff_all'].qcut(x, q, labels=None, retbins=False, precision=3, duplicates='raise')

In [126]:
ml['Loan_Status'].value_counts(normalize=True)

Loan_Status
1    0.687296
0    0.312704
Name: proportion, dtype: float64

In [127]:
ml.describe()

,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,min_installment,ratio_of_income,diff_all
count,599.000000,614.000000,614.000000,592.000000,600.00000,564.000000,614.000000,578.000000,578.000000,578.000000
mean,0.762938,5403.459283,1621.245798,146.412162,342.00000,0.842199,0.687296,384.239880,10.120997,6691.381089
std,1.015216,6109.041673,2926.248369,85.587325,65.12041,0.364878,0.463973,420.366948,20.051338,6455.041863
min,0.000000,150.000000,0.000000,9.000000,12.00000,0.000000,0.000000,20.000000,0.670253,36.555556
25%,0.000000,2877.500000,0.000000,100.000000,360.00000,1.000000,0.000000,228.888889,5.484486,3863.527778
50%,0.000000,3812.500000,1188.500000,128.000000,360.00000,1.000000,1.000000,292.222222,7.074611,5046.388889
75%,2.000000,5795.000000,2297.250000,168.000000,360.00000,1.000000,1.000000,411.111111,10.023864,7052.222222
max,3.000000,81000.000000,41667.000000,700.000000,480.00000,1.000000,1.000000,7400.000000,373.931624,80200.000000


In [128]:
ml[ml['ratio_of_income']>=100]

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,min_installment,ratio_of_income,diff_all
216,Male,Yes,0.0,Graduate,No,150,1800.0,135.0,360.0,1.0,Rural,0,300.000000,200.000000,1650.000000
468,Female,Yes,2.0,Not Graduate,NaN,210,2917.0,98.0,360.0,1.0,Semiurban,1,217.777778,103.703704,2909.222222
497,Male,Yes,0.0,Graduate,No,4625,2857.0,111.0,12.0,NaN,Urban,1,7400.000000,160.000000,82.000000
600,Female,No,3.0,Graduate,NaN,416,41667.0,350.0,180.0,NaN,Urban,0,1555.555556,373.931624,40527.444444


In [129]:
ml[ml['ratio_of_income']>=50]

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,min_installment,ratio_of_income,diff_all
188,NaN,Yes,0.0,Graduate,Yes,674,5296.0,168.0,360.0,1.0,Rural,1,373.333333,55.390702,5596.666667
216,Male,Yes,0.0,Graduate,No,150,1800.0,135.0,360.0,1.0,Rural,0,300.000000,200.000000,1650.000000
242,Male,No,0.0,Graduate,No,2965,5701.0,155.0,60.0,1.0,Urban,1,2066.666667,69.702080,6599.333333
262,Female,No,1.0,Graduate,No,3481,0.0,155.0,36.0,1.0,Semiurban,0,3444.444444,98.949855,36.555556
468,Female,Yes,2.0,Not Graduate,NaN,210,2917.0,98.0,360.0,1.0,Semiurban,1,217.777778,103.703704,2909.222222
497,Male,Yes,0.0,Graduate,No,4625,2857.0,111.0,12.0,NaN,Urban,1,7400.000000,160.000000,82.000000
546,Male,No,0.0,Not Graduate,No,3358,0.0,80.0,36.0,1.0,Semiurban,0,1777.777778,52.941566,1580.222222
600,Female,No,3.0,Graduate,NaN,416,41667.0,350.0,180.0,NaN,Urban,0,1555.555556,373.931624,40527.444444


In [130]:
ml.describe()

,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,min_installment,ratio_of_income,diff_all
count,599.000000,614.000000,614.000000,592.000000,600.00000,564.000000,614.000000,578.000000,578.000000,578.000000
mean,0.762938,5403.459283,1621.245798,146.412162,342.00000,0.842199,0.687296,384.239880,10.120997,6691.381089
std,1.015216,6109.041673,2926.248369,85.587325,65.12041,0.364878,0.463973,420.366948,20.051338,6455.041863
min,0.000000,150.000000,0.000000,9.000000,12.00000,0.000000,0.000000,20.000000,0.670253,36.555556
25%,0.000000,2877.500000,0.000000,100.000000,360.00000,1.000000,0.000000,228.888889,5.484486,3863.527778
50%,0.000000,3812.500000,1188.500000,128.000000,360.00000,1.000000,1.000000,292.222222,7.074611,5046.388889
75%,2.000000,5795.000000,2297.250000,168.000000,360.00000,1.000000,1.000000,411.111111,10.023864,7052.222222
max,3.000000,81000.000000,41667.000000,700.000000,480.00000,1.000000,1.000000,7400.000000,373.931624,80200.000000


In [131]:
#ml[ml['diff']>7000]

In [132]:
#ml[ml['diff']<500]

In [133]:
ml[ml['ApplicantIncome']<500]

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,min_installment,ratio_of_income,diff_all
216,Male,Yes,0.0,Graduate,No,150,1800.0,135.0,360.0,1.0,Rural,0,300.000000,200.000000,1650.000000
468,Female,Yes,2.0,Not Graduate,NaN,210,2917.0,98.0,360.0,1.0,Semiurban,1,217.777778,103.703704,2909.222222
600,Female,No,3.0,Graduate,NaN,416,41667.0,350.0,180.0,NaN,Urban,0,1555.555556,373.931624,40527.444444


In [134]:
ml[ml['min_installment']>2000]

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,min_installment,ratio_of_income,diff_all
155,Male,Yes,3.0,Graduate,No,39999,0.0,600.0,180.0,0.0,Semiurban,1,2666.666667,6.666833,37332.333333
242,Male,No,0.0,Graduate,No,2965,5701.0,155.0,60.0,1.0,Urban,1,2066.666667,69.702080,6599.333333
262,Female,No,1.0,Graduate,No,3481,0.0,155.0,36.0,1.0,Semiurban,0,3444.444444,98.949855,36.555556
333,Male,Yes,0.0,Graduate,NaN,63337,0.0,490.0,180.0,1.0,Urban,1,2177.777778,3.438397,61159.222222
497,Male,Yes,0.0,Graduate,No,4625,2857.0,111.0,12.0,NaN,Urban,1,7400.000000,160.000000,82.000000


In [135]:
ml[ml['Loan_Amount_Term']<=36]

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,min_installment,ratio_of_income,diff_all
262,Female,No,1.0,Graduate,No,3481,0.0,155.0,36.0,1.0,Semiurban,0,3444.444444,98.949855,36.555556
497,Male,Yes,0.0,Graduate,No,4625,2857.0,111.0,12.0,NaN,Urban,1,7400.000000,160.000000,82.000000
546,Male,No,0.0,Not Graduate,No,3358,0.0,80.0,36.0,1.0,Semiurban,0,1777.777778,52.941566,1580.222222


In [136]:
ml.corr(numeric_only=True)

,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,min_installment,ratio_of_income,diff_all
Dependents,1.000000,0.118679,0.027259,0.163997,-0.100484,-0.050082,0.006781,0.100016,0.059466,0.129358
ApplicantIncome,0.118679,1.000000,-0.116605,0.570909,-0.045306,-0.014715,-0.004710,0.323509,-0.133009,0.891278
CoapplicantIncome,0.027259,-0.116605,1.000000,0.188619,-0.059878,-0.002056,-0.059187,0.135826,0.519393,0.345198
LoanAmount,0.163997,0.570909,0.188619,1.000000,0.039447,-0.008433,-0.037318,0.489983,0.087581,0.606992
Loan_Amount_Term,-0.100484,-0.045306,-0.059878,0.039447,1.000000,0.001470,-0.021268,-0.500726,-0.296254,-0.040813
Credit_History,-0.050082,-0.014715,-0.002056,-0.008433,0.001470,1.000000,0.561678,-0.009747,0.009736,-0.027402
Loan_Status,0.006781,-0.004710,-0.059187,-0.037318,-0.021268,0.561678,1.000000,-0.015394,-0.077971,-0.036995
min_installment,0.100016,0.323509,0.135826,0.489983,-0.500726,-0.009747,-0.015394,1.000000,0.438563,0.310061
ratio_of_income,0.059466,-0.133009,0.519393,0.087581,-0.296254,0.009736,-0.077971,0.438563,1.000000,0.083929
diff_all,0.129358,0.891278,0.345198,0.606992,-0.040813,-0.027402,-0.036995,0.310061,0.083929,1.000000


In [137]:
px.imshow(ml.corr(numeric_only=True),text_auto=True , width=1000,height=1200)

In [138]:
ml.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status',
       'min_installment', 'ratio_of_income', 'diff_all'],
      dtype='object')

In [139]:
numerical = [ 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'installment', 'ratio_of_income', 'ratio_of_all_incomes', 'diff',
       'diff_all']

In [140]:
categorical = ['Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed',  'Credit_History', 'Property_Area', 'Loan_Status']

In [141]:
discrete = [ 'Dependents','Credit_History','Loan_Amount_Term']

# Modeling

##### splitting X,y target and features

In [142]:
X = ml.drop('Loan_Status',axis=1)

In [143]:
y = ml['Loan_Status']

In [144]:
#mode_impute = ['Gender','Married','Self_Employed']

In [145]:
#knn_impute = ['ApplicantIncome', 'CoapplicantIncome','LoanAmount','Loan_Amount_Term','installment','ratio_of_income','ratio_of_all_incomes','diff','diff_all']

In [146]:
#median_imputer = ['Dependents','Credit_History']

In [147]:
#onh = ['Gender','Married','Education','Self_Employed','Property_Area']

In [148]:
#ordinal= ['Dependents']

In [149]:
cat_cols = X.select_dtypes(include='object').columns.to_list()
cat_cols

['Gender', 'Married', 'Education', 'Self_Employed', 'Property_Area']

In [150]:
cat_cols.append('Dependents')

In [151]:
cat_cols.append('Credit_History')

In [152]:
#cat_cols.append('Loan_Amount_Term')

In [153]:
cat_cols

['Gender',
 'Married',
 'Education',
 'Self_Employed',
 'Property_Area',
 'Dependents',
 'Credit_History']

In [154]:
scaling_col = X.select_dtypes(exclude='object').columns.to_list()
scaling_col

['Dependents',
 'ApplicantIncome',
 'CoapplicantIncome',
 'LoanAmount',
 'Loan_Amount_Term',
 'Credit_History',
 'min_installment',
 'ratio_of_income',
 'diff_all']

In [155]:
X.describe()

,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,min_installment,ratio_of_income,diff_all
count,599.000000,614.000000,614.000000,592.000000,600.00000,564.000000,578.000000,578.000000,578.000000
mean,0.762938,5403.459283,1621.245798,146.412162,342.00000,0.842199,384.239880,10.120997,6691.381089
std,1.015216,6109.041673,2926.248369,85.587325,65.12041,0.364878,420.366948,20.051338,6455.041863
min,0.000000,150.000000,0.000000,9.000000,12.00000,0.000000,20.000000,0.670253,36.555556
25%,0.000000,2877.500000,0.000000,100.000000,360.00000,1.000000,228.888889,5.484486,3863.527778
50%,0.000000,3812.500000,1188.500000,128.000000,360.00000,1.000000,292.222222,7.074611,5046.388889
75%,2.000000,5795.000000,2297.250000,168.000000,360.00000,1.000000,411.111111,10.023864,7052.222222
max,3.000000,81000.000000,41667.000000,700.000000,480.00000,1.000000,7400.000000,373.931624,80200.000000


In [156]:
scaling_col.remove('Dependents')

In [157]:
scaling_col.remove('Credit_History')

In [158]:
#scaling_col.remove('Loan_Amount_Term')

In [159]:
#discrete_col = ['Dependents','Credit_History','Loan_Amount_Term']

In [160]:
scaling_col

['ApplicantIncome',
 'CoapplicantIncome',
 'LoanAmount',
 'Loan_Amount_Term',
 'min_installment',
 'ratio_of_income',
 'diff_all']

In [161]:
cat_cols

['Gender',
 'Married',
 'Education',
 'Self_Employed',
 'Property_Area',
 'Dependents',
 'Credit_History']

In [162]:
X.isna().sum()

Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
min_installment      36
ratio_of_income      36
diff_all             36
dtype: int64

In [163]:
X

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,min_installment,ratio_of_income,diff_all
0,Male,No,0.0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,NaN,NaN,NaN
1,Male,Yes,1.0,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,284.444444,6.206512,5806.555556
2,Male,Yes,0.0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,146.666667,4.888889,2853.333333
3,Male,Yes,0.0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,266.666667,10.323913,4674.333333
4,Male,No,0.0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,313.333333,5.222222,5686.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,Female,No,0.0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,157.777778,5.440613,2742.222222
610,Male,Yes,3.0,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,177.777778,4.329707,3928.222222
611,Male,Yes,1.0,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,562.222222,6.965092,7749.777778
612,Male,Yes,2.0,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,415.555556,5.480094,7167.444444


# Modeling

### Detecting which model will give best scores (specially : precision& accuracy )

In [164]:
# metrics and scores names
sklearn.metrics.get_scorer_names()

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_negative_likelihood_ratio',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'positive_likelihood_ratio',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',

In [165]:
scalers = [
    ('std scaler:', StandardScaler()), 
    ('MinMax scaler :', MinMaxScaler()), 
    ('Robust scaler: ', RobustScaler())
]

In [185]:
#models
models = [
    ('KNN', KNeighborsClassifier()),
    ('Naive', GaussianNB()),
    ('DTC', DecisionTreeClassifier()),
    ('RFC', RandomForestClassifier()),
    ('ADA' ,AdaBoostClassifier()),
    ('GB',GradientBoostingClassifier()),
    ('LogisticR',LogisticRegression()),
    ('SVM', SVC()),      

    ('XGB', XGBClassifier())]
    

In [192]:
result_models=[] # to append data of scores
for scaler_name, scaler in scalers:
    # print scaler name
    print(f'For {scaler_name} :')
    print('='*30)
    
    #pipelines for combinations coluns
    cat_preprocessing = Pipeline(steps=[
        ('simple_imputer_mode', SimpleImputer(strategy='most_frequent') ),
        ('onh_encoder', OneHotEncoder(sparse_output=False, drop='first') ),
        (scaler_name,scaler)

    ])
    
    num_preprocessing = Pipeline(steps=[
        ('knn_imputer', KNNImputer() ),
        (scaler_name,scaler)
    ])
    
    # Create the ColumnTransformer preprocessing
    preprocessor =ColumnTransformer(
        transformers=[
           ('cat_preprocessing', cat_preprocessing , cat_cols)
            ,('num_preprocessing', num_preprocessing , scaling_col)
            #,('discrete_preprocessing', discrete_preprocessing , discrete_col)
            #,('OverSampling',SMOTENC(categorical_features=(cat_cols+discrete_col)),ml.columns)
            ],
        remainder='passthrough'
         )
   
    for model_name, model in models:
        # print model name
        print(f"For {model_name}:")
        # Steps Creation
        stepss = list()
        
        stepss.append(('preprocessor', preprocessor))
        stepss.append(('overSampling',SVMSMOTE()))
        stepss.append((model_name, model))
        
        # Create the pipeline
        pipeline = Pipeline(steps=stepss)
        
        ## scoring
        # Perform cross-validation
        # Example evaluation metrics
        cv_results = cross_validate(pipeline, X, y, scoring={'roc_auc': 'roc_auc',
                                                             'accuracy':'accuracy',
                                                             'recall':'recall',
                                                             'precision':'precision',
                                                             'f1':'f1'}# interested scors and metrics
                                                             
                                    , cv=5, return_train_score=True,n_jobs=-1,error_score='raise')  # X and y are your input data and targets
        
        
        # Access the results
        test_roc_auc = cv_results['test_roc_auc'].mean()
        train_roc_auc = cv_results['train_roc_auc'].mean()
        print("test_roc_auc :", test_roc_auc)
        print("train_roc_auc:", train_roc_auc)
        print('-'*50)
        test_accuracy = cv_results['test_accuracy'].mean()
        train_accuracy = cv_results['train_accuracy'].mean()
        print("test_accuracy :", test_accuracy)
        print("train_accuracy:", train_accuracy)
        print('-'*50)
        test_recall = cv_results['test_recall'].mean()
        train_recall = cv_results['train_recall'].mean()
        print("test_recall :", test_recall)
        print("train_recall:", train_recall)
        print('-'*50)
        test_precision = cv_results['test_precision'].mean()
        train_precision = cv_results['train_precision'].mean()
        print("test_precision :", test_precision)
        print("train_precision:", train_precision)
        print('-'*50)
        test_f1 = cv_results['test_f1'].mean()
        train_f1 = cv_results['train_f1'].mean()
        print("test_f1 :", test_f1)
        print("train_f1:", train_f1)
        result_models.append([test_roc_auc,train_roc_auc,test_accuracy,train_accuracy, test_recall , train_recall ,
        test_precision,train_precision,test_f1,train_f1])
        print('*' * 50)
    print('*' * 100)
    print('_' * 100)

For std scaler: :
For KNN:
test_roc_auc : 0.7146005473714452
train_roc_auc: 0.9070247067897348
--------------------------------------------------
test_accuracy : 0.7084232973477276
train_accuracy: 0.8395757786498435
--------------------------------------------------
test_recall : 0.7890756302521009
train_recall: 0.8833318701385353
--------------------------------------------------
test_precision : 0.7877409387675639
train_precision: 0.8839284553073155
--------------------------------------------------
test_f1 : 0.7879393210373824
train_f1: 0.8831312441758461
**************************************************
For Naive:
test_roc_auc : 0.7487309714709096
train_roc_auc: 0.7733960768998431
--------------------------------------------------
test_accuracy : 0.781713981074237
train_accuracy: 0.8029324590598247
--------------------------------------------------
test_recall : 0.9170308123249299
train_recall: 0.9478622724000492
--------------------------------------------------
test_precision : 

test_roc_auc : 0.7571963846267252
train_roc_auc: 0.9759494179586958
--------------------------------------------------
test_accuracy : 0.765480474476876
train_accuracy: 0.9267150166410014
--------------------------------------------------
test_recall : 0.8672829131652661
train_recall: 0.9828209225150563
--------------------------------------------------
test_precision : 0.8079889117397114
train_precision: 0.9166970782023662
--------------------------------------------------
test_f1 : 0.835641582635024
train_f1: 0.948567085138527
**************************************************
For LogisticR:
test_roc_auc : 0.768964246967343
train_roc_auc: 0.7998293348515565
--------------------------------------------------
test_accuracy : 0.7572304411568705
train_accuracy: 0.785439537694766
--------------------------------------------------
test_recall : 0.869579831932773
train_recall: 0.8963267957789757
--------------------------------------------------
test_precision : 0.7958618439937093
train_pre

In [193]:
# result_models to data frame to be clear
results = pd.DataFrame(data=result_models,index=['KNN_std','Naive_std','DTC_std','RFC_std','ADA_std' ,'GB_std','LogisticR_std','SVM_std','XGB_std',
    'KNN_minmax','Naive_minmax','DTC_minmax','RFC_minmax','ADA_minmax' ,'GB_minmax','LogisticR_minmax','SVM_minmax','XGB_minmax',
    'KNN_robust','Naive_robust','DTC_robust','RFC_robust','ADA_robust' ,'GB_robust','LogisticR_robust','SVM_robust','XGB_robust'],
            columns=['test_roc_auc','train_roc_auc','test_accuracy','train_accuracy', 'test_recall' , 'train_recall' ,
                     'test_precision','train_precision','test_f1','train_f1'])

In [194]:
results #SVCSMOTE

,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy,test_recall,train_recall,test_precision,train_precision,test_f1,train_f1
KNN_std,0.714601,0.907025,0.708423,0.839576,0.789076,0.883332,0.787741,0.883928,0.787939,0.883131
Naive_std,0.748731,0.773396,0.781714,0.802932,0.917031,0.947862,0.796991,0.801665,0.852210,0.868625
DTC_std,0.678581,1.000000,0.705131,1.000000,0.748796,1.000000,0.809917,1.000000,0.777110,1.000000
RFC_std,0.769293,1.000000,0.760576,1.000000,0.872045,1.000000,0.798643,1.000000,0.833673,1.000000
ADA_std,0.730373,0.898527,0.745835,0.844059,0.833950,0.916487,0.806520,0.864822,0.817711,0.889758
GB_std,0.743474,0.971892,0.763868,0.918977,0.876751,0.980454,0.801109,0.908870,0.836367,0.943295
LogisticR_std,0.761540,0.801270,0.740944,0.774434,0.829216,0.859604,0.801691,0.820849,0.813703,0.839501
SVM_std,0.738137,0.892226,0.772011,0.844868,0.900476,0.944330,0.795779,0.847895,0.844603,0.893020
XGB_std,0.748492,1.000000,0.744302,1.000000,0.831681,1.000000,0.804945,1.000000,0.817466,1.000000
KNN_minmax,0.727443,0.901708,0.732840,0.833465,0.841008,0.893382,0.785808,0.868635,0.811858,0.880587


In [201]:
# Initialize RFE feature selector
RFE_selector = RFE(LogisticRegression(), n_features_to_select=10) 

result_models=[] # to append data of scores
for scaler_name, scaler in scalers:
    # print scaler name
    print(f'For {scaler_name} :')
    print('='*30)
    
    #pipelines for combinations coluns
    cat_preprocessing = Pipeline(steps=[
        ('simple_imputer_mode', SimpleImputer(strategy='most_frequent') ),
        ('onh_encoder', OneHotEncoder(sparse_output=False, drop='first') ),
        (scaler_name,scaler)

    ])
    
    num_preprocessing = Pipeline(steps=[
        ('knn_imputer', KNNImputer() ),
        (scaler_name,scaler)
    ])
    
    # Create the ColumnTransformer preprocessing
    preprocessor =ColumnTransformer(
        transformers=[
           ('cat_preprocessing', cat_preprocessing , cat_cols)
            ,('num_preprocessing', num_preprocessing , scaling_col)
         
            ],
        remainder='passthrough'
         )
   
    for model_name, model in models:
        # print model name
        print(f"For {model_name}:")
        # Steps Creation
        stepss = list()
        
        stepss.append(('preprocessor', preprocessor))
        stepss.append(('overSampling',SVMSMOTE()))
        stepss.append(('feature_selection_RFE',RFE_selector))
        stepss.append((model_name, model))
        
        # Create the pipeline
        pipeline = Pipeline(steps=stepss)
        
        ## scoring
        # Perform cross-validation
        # Example evaluation metrics
        cv_results = cross_validate(pipeline, X, y, scoring={'roc_auc': 'roc_auc',
                                                             'accuracy':'accuracy',
                                                             'recall':'recall',
                                                             'precision':'precision',
                                                             'f1':'f1'}# interested scors and metrics
                                                             
                                    , cv=5, return_train_score=True,n_jobs=-1,error_score='raise')  # X and y are your input data and targets
        
        
        # Access the results
        test_roc_auc = cv_results['test_roc_auc'].mean()
        train_roc_auc = cv_results['train_roc_auc'].mean()
        print("test_roc_auc :", test_roc_auc)
        print("train_roc_auc:", train_roc_auc)
        print('-'*50)
        test_accuracy = cv_results['test_accuracy'].mean()
        train_accuracy = cv_results['train_accuracy'].mean()
        print("test_accuracy :", test_accuracy)
        print("train_accuracy:", train_accuracy)
        print('-'*50)
        test_recall = cv_results['test_recall'].mean()
        train_recall = cv_results['train_recall'].mean()
        print("test_recall :", test_recall)
        print("train_recall:", train_recall)
        print('-'*50)
        test_precision = cv_results['test_precision'].mean()
        train_precision = cv_results['train_precision'].mean()
        print("test_precision :", test_precision)
        print("train_precision:", train_precision)
        print('-'*50)
        test_f1 = cv_results['test_f1'].mean()
        train_f1 = cv_results['train_f1'].mean()
        print("test_f1 :", test_f1)
        print("train_f1:", train_f1)
        result_models.append([test_roc_auc,train_roc_auc,test_accuracy,train_accuracy, test_recall , train_recall ,
        test_precision,train_precision,test_f1,train_f1])
        print('*' * 50)
    print('*' * 100)
    print('_' * 100)

For std scaler: :
For KNN:
test_roc_auc : 0.7222877707088233
train_roc_auc: 0.9088129574474353
--------------------------------------------------
test_accuracy : 0.7117153138744502
train_accuracy: 0.8403904426009637
--------------------------------------------------
test_recall : 0.8033333333333333
train_recall: 0.8880410162765789
--------------------------------------------------
test_precision : 0.7848184120779402
train_precision: 0.8809885680974633
--------------------------------------------------
test_f1 : 0.7928506299469011
train_f1: 0.884205951871637
**************************************************
For Naive:
test_roc_auc : 0.7553180273458912
train_roc_auc: 0.7876731711163817
--------------------------------------------------
test_accuracy : 0.7866186858589896
train_accuracy: 0.8045642706936235
--------------------------------------------------
test_recall : 0.9384033613445378
train_recall: 0.9549751549523291
--------------------------------------------------
test_precision : 

test_roc_auc : 0.7248079663714339
train_roc_auc: 0.9643852062410814
--------------------------------------------------
test_accuracy : 0.757297081167533
train_accuracy: 0.904323348732469
--------------------------------------------------
test_recall : 0.8694117647058823
train_recall: 0.976902006917985
--------------------------------------------------
test_precision : 0.7984903739849178
train_precision: 0.8937927734938743
--------------------------------------------------
test_f1 : 0.8308568673402608
train_f1: 0.9334944179852072
**************************************************
For LogisticR:
test_roc_auc : 0.7662530761292372
train_roc_auc: 0.7959560666309058
--------------------------------------------------
test_accuracy : 0.7719845395175262
train_accuracy: 0.7846182504594903
--------------------------------------------------
test_recall : 0.8814005602240895
train_recall: 0.8904307060207539
--------------------------------------------------
test_precision : 0.806056812848513
train_p

In [202]:
# result_models to data frame to be clear
results = pd.DataFrame(data=result_models,index=['KNN_std','Naive_std','DTC_std','RFC_std','ADA_std' ,'GB_std','LogisticR_std','SVM_std','XGB_std',
    'KNN_minmax','Naive_minmax','DTC_minmax','RFC_minmax','ADA_minmax' ,'GB_minmax','LogisticR_minmax','SVM_minmax','XGB_minmax',
    'KNN_robust','Naive_robust','DTC_robust','RFC_robust','ADA_robust' ,'GB_robust','LogisticR_robust','SVM_robust','XGB_robust'],
            columns=['test_roc_auc','train_roc_auc','test_accuracy','train_accuracy', 'test_recall' , 'train_recall' ,
                     'test_precision','train_precision','test_f1','train_f1'])

In [203]:
results #SVCSMOTE

,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy,test_recall,train_recall,test_precision,train_precision,test_f1,train_f1
KNN_std,0.722288,0.908813,0.711715,0.840390,0.803333,0.888041,0.784818,0.880989,0.792851,0.884206
Naive_std,0.755318,0.787673,0.786619,0.804564,0.938403,0.954975,0.791049,0.799713,0.857975,0.870417
DTC_std,0.659177,1.000000,0.693816,1.000000,0.751148,1.000000,0.792812,1.000000,0.771130,1.000000
RFC_std,0.739334,1.000000,0.750766,1.000000,0.860112,1.000000,0.795327,1.000000,0.826001,1.000000
ADA_std,0.731819,0.872369,0.749100,0.829809,0.843417,0.912948,0.803733,0.850368,0.821474,0.880372
GB_std,0.729030,0.949345,0.739424,0.891699,0.848151,0.974537,0.790593,0.880773,0.817188,0.925203
LogisticR_std,0.758328,0.799277,0.749100,0.785420,0.845826,0.882712,0.800692,0.819114,0.821741,0.849682
SVM_std,0.706444,0.845493,0.762255,0.819212,0.896050,0.932418,0.786024,0.829320,0.835188,0.875716
XGB_std,0.735843,0.999996,0.716620,0.998778,0.796331,1.000000,0.793647,0.998227,0.794708,0.999112
KNN_minmax,0.721699,0.901859,0.732933,0.832245,0.827003,0.886859,0.794224,0.871560,0.809519,0.879040


In [199]:
results #SVCSMOTE

,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy,test_recall,train_recall,test_precision,train_precision,test_f1,train_f1
KNN_std,0.729777,0.905057,0.723111,0.835502,0.812745,0.885674,0.790820,0.876577,0.801305,0.881022
Naive_std,0.763441,0.782297,0.799680,0.801713,0.950196,0.954375,0.798498,0.797286,0.867079,0.868646
DTC_std,0.644745,1.000000,0.684046,1.000000,0.748599,1.000000,0.785270,1.000000,0.764642,1.000000
RFC_std,0.760276,1.000000,0.755604,1.000000,0.869608,1.000000,0.794886,1.000000,0.829856,1.000000
ADA_std,0.753873,0.878194,0.762135,0.829803,0.860028,0.904040,0.809609,0.856676,0.831590,0.879571
GB_std,0.740382,0.953466,0.750740,0.893723,0.864790,0.978084,0.792946,0.880856,0.826539,0.926799
LogisticR_std,0.756370,0.798352,0.747408,0.780948,0.848179,0.876194,0.796597,0.818869,0.820270,0.845856
SVM_std,0.707959,0.848293,0.739398,0.822470,0.864846,0.927143,0.781078,0.833733,0.819412,0.877542
XGB_std,0.723495,0.999998,0.721431,0.999593,0.796134,1.000000,0.799983,0.999408,0.797516,0.999704
KNN_minmax,0.714959,0.900734,0.731281,0.837136,0.831821,0.914091,0.789950,0.858272,0.809684,0.885221


## HyperParameter Tuning 

In [206]:
# Initialize RFE feature selector
RFE_selector = RFE(LogisticRegression(), n_features_to_select=10) 

cat_preprocessing = Pipeline(steps=[
    ('simple_imputer_mode', SimpleImputer(strategy='most_frequent') ),
    ('onh_encoder', OneHotEncoder(sparse_output=False, drop='first') ),
    ('RobustScaler',RobustScaler())

])

num_preprocessing = Pipeline(steps=[
    ('knn_imputer', KNNImputer() ),
    ('RobustScaler',RobustScaler())
])

# Create the ColumnTransformer preprocessing
preprocessor =ColumnTransformer(
    transformers=[
       ('cat_preprocessing', cat_preprocessing , cat_cols)
        ,('num_preprocessing', num_preprocessing , scaling_col)
        #,('discrete_preprocessing', discrete_preprocessing , discrete_col)
        ],
    remainder='passthrough'
     )
# Steps Creation
stepss = list()

stepss.append(('preprocessor', preprocessor))
stepss.append(('overSampling',SVMSMOTE()))
stepss.append(('feature_selection_RFE',RFE_selector))

stepss.append(('LR_model', LogisticRegression()))

# Create the pipeline
pipeline = Pipeline(steps=stepss)

## scoring
# Perform cross-validation
# Example evaluation metrics
cv_results = cross_validate(pipeline, X, y, scoring={'roc_auc': 'roc_auc',
                                                             'accuracy':'accuracy',
                                                             'recall':'recall',
                                                             'precision':'precision',
                                                             'f1':'f1'}# interested scors and metrics
                                                             
                                    , cv=5, return_train_score=True,n_jobs=-1,error_score='raise')  # X and y are your input data and targets
        
        
# Access the results
test_roc_auc = cv_results['test_roc_auc'].mean()
train_roc_auc = cv_results['train_roc_auc'].mean()
print("test_roc_auc :", test_roc_auc)
print("train_roc_auc:", train_roc_auc)
print('-'*50)
test_accuracy = cv_results['test_accuracy'].mean()
train_accuracy = cv_results['train_accuracy'].mean()
print("test_accuracy :", test_accuracy)
print("train_accuracy:", train_accuracy)
print('-'*50)
test_recall = cv_results['test_recall'].mean()
train_recall = cv_results['train_recall'].mean()
print("test_recall :", test_recall)
print("train_recall:", train_recall)
print('-'*50)
test_precision = cv_results['test_precision'].mean()
train_precision = cv_results['train_precision'].mean()
print("test_precision :", test_precision)
print("train_precision:", train_precision)
print('-'*50)
test_f1 = cv_results['test_f1'].mean()
train_f1 = cv_results['train_f1'].mean()
print("test_f1 :", test_f1)
print("train_f1:", train_f1)
result_models.append([test_roc_auc,train_roc_auc,test_accuracy,train_accuracy, test_recall , train_recall ,
test_precision,train_precision,test_f1,train_f1])
print('*' * 50)

test_roc_auc : 0.764537474908992
train_roc_auc: 0.7928674171524313
--------------------------------------------------
test_accuracy : 0.7573370651739304
train_accuracy: 0.7821709469640522
--------------------------------------------------
test_recall : 0.8647619047619047
train_recall: 0.8969606517654908
--------------------------------------------------
test_precision : 0.8000133428279602
train_precision: 0.8075820976811903
--------------------------------------------------
test_f1 : 0.8298489277186428
train_f1: 0.8496502741928568
**************************************************


In [207]:
param_grid = {
    'LR_model__solver':['newton-cg', 'lbfgs', 'liblinear'],#'newton-cholesky'] ,
    'LR_model__penalty':['l2'],#,'l1',None],
    'LR_model__C':[100, 10, 1.0, 0.1, 0.01]#[.9,.5,.09, 0.1, 0.2,.3,.4,1,10,100]


}

In [208]:
# Create GridSearchCV instance
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, scoring='f1'
                                                             , n_jobs=-1, return_train_score=True)

# Fit the pipeline with GridSearch to the data
grid_search.fit(X, y)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'LR_model__C': 0.1, 'LR_model__penalty': 'l2', 'LR_model__solver': 'newton-cg'}
Best Score: 0.8535016021612641


In [209]:
# Get the mean test score and mean train score for the best estimator
mean_test_score = grid_search.cv_results_['mean_test_score'][grid_search.best_index_]
mean_train_score = grid_search.cv_results_['mean_train_score'][grid_search.best_index_]

print("Mean Test Score:", mean_test_score)
print("Mean Train Score:", mean_train_score)

Mean Test Score: 0.8535016021612641
Mean Train Score: 0.8581514726212225


In [85]:
'''# Create GridSearchCV instance
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, scoring='precision'
                                                             , n_jobs=-1, return_train_score=True)

# Fit the pipeline with GridSearch to the data
grid_search.fit(X, y)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)'''

Best Parameters: {'LR_model__C': 10, 'LR_model__penalty': 'l2', 'LR_model__solver': 'liblinear'}
Best Score: 0.8149136400662058


In [210]:
grid_search.best_estimator_

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat_preprocessing',
                                                  Pipeline(steps=[('simple_imputer_mode',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onh_encoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 sparse_output=False)),
                                                                  ('RobustScaler',
                                                                   RobustScaler())]),
                                                  ['Gender', 'Married',
                                                   'Education', 'Self_Employed',
                                                   'Property_Area'...
                                                  Pipeline(steps=[('knn_imputer',
                                                                   KNNImputer()),
                                                                  ('RobustScaler',
                                                                   RobustScaler())]),
                                                  ['ApplicantIncome',
                                                   'CoapplicantIncome',
                                                   'LoanAmount',
                                                   'Loan_Amount_Term',
                                                   'min_installment',
                                                   'ratio_of_income',
                                                   'diff_all'])])),
                ('overSampling', SVMSMOTE()),
                ('feature_selection_RFE',
                 RFE(estimator=LogisticRegression(), n_features_to_select=10)),
                ('LR_model', LogisticRegression(C=0.1, solver='newton-cg'))])

# Saving model with its best parameters

In [211]:
Model = grid_search.best_estimator_

In [212]:
Model.fit(X,y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat_preprocessing',
                                                  Pipeline(steps=[('simple_imputer_mode',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onh_encoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 sparse_output=False)),
                                                                  ('RobustScaler',
                                                                   RobustScaler())]),
                                                  ['Gender', 'Married',
                                                   'Education', 'Self_Employed',
                                                   'Property_Area'...
                                                  Pipeline(steps=[('knn_imputer',
                                                                   KNNImputer()),
                                                                  ('RobustScaler',
                                                                   RobustScaler())]),
                                                  ['ApplicantIncome',
                                                   'CoapplicantIncome',
                                                   'LoanAmount',
                                                   'Loan_Amount_Term',
                                                   'min_installment',
                                                   'ratio_of_income',
                                                   'diff_all'])])),
                ('overSampling', SVMSMOTE()),
                ('feature_selection_RFE',
                 RFE(estimator=LogisticRegression(), n_features_to_select=10)),
                ('LR_model', LogisticRegression(C=0.1, solver='newton-cg'))])

# scoring best params

In [213]:
cat_preprocessing = Pipeline(steps=[
    ('simple_imputer_mode', SimpleImputer(strategy='most_frequent') ),
    ('onh_encoder', OneHotEncoder(sparse_output=False, drop='first') ),
    ('RobustScaler',RobustScaler())
])

num_preprocessing = Pipeline(steps=[
    ('knn_imputer', KNNImputer() ),
    ('RobustScaler',RobustScaler())
])

# Create the ColumnTransformer preprocessing
preprocessor =ColumnTransformer(
    transformers=[
       ('cat_preprocessing', cat_preprocessing , cat_cols)
        ,('num_preprocessing', num_preprocessing , scaling_col)
        ],
    remainder='passthrough'
     )
# Steps Creation
stepss = list()

stepss.append(('preprocessor', preprocessor))
stepss.append(('overSampling',SVMSMOTE()))
stepss.append(('feature_selection_RFE',RFE_selector))
stepss.append(('LR_model', LogisticRegression(C=.1 , penalty= 'l2', solver= 'newton-cg')))

# Create the pipeline
pipeline = Pipeline(steps=stepss)

## scoring
# Perform cross-validation
# Example evaluation metrics
cv_results = cross_validate(pipeline, X, y, scoring={'roc_auc': 'roc_auc',
                                                             'accuracy':'accuracy',
                                                             'recall':'recall',
                                                             'precision':'precision',
                                                             'f1':'f1'}# interested scors and metrics
                                                             
                                    , cv=5, return_train_score=True,n_jobs=-1,error_score='raise')  # X and y are your input data and targets
        
# Access the results
test_roc_auc = cv_results['test_roc_auc'].mean()
train_roc_auc = cv_results['train_roc_auc'].mean()
print("test_roc_auc :", test_roc_auc)
print("train_roc_auc:", train_roc_auc)
print('-'*50)
test_accuracy = cv_results['test_accuracy'].mean()
train_accuracy = cv_results['train_accuracy'].mean()
print("test_accuracy :", test_accuracy)
print("train_accuracy:", train_accuracy)
print('-'*50)
test_recall = cv_results['test_recall'].mean()
train_recall = cv_results['train_recall'].mean()
print("test_recall :", test_recall)
print("train_recall:", train_recall)
print('-'*50)
test_precision = cv_results['test_precision'].mean()
train_precision = cv_results['train_precision'].mean()
print("test_precision :", test_precision)
print("train_precision:", train_precision)
print('-'*50)
test_f1 = cv_results['test_f1'].mean()
train_f1 = cv_results['train_f1'].mean()
print("test_f1 :", test_f1)
print("train_f1:", train_f1)
result_models.append([test_roc_auc,train_roc_auc,test_accuracy,train_accuracy, test_recall , train_recall ,
test_precision,train_precision,test_f1,train_f1])
print('*' * 50)

test_roc_auc : 0.7612493715434893
train_roc_auc: 0.7888691823339433
--------------------------------------------------
test_accuracy : 0.7686791949886712
train_accuracy: 0.7862401271670558
--------------------------------------------------
test_recall : 0.8908683473389356
train_recall: 0.9040138359700103
--------------------------------------------------
test_precision : 0.7982539607325138
train_precision: 0.8079794261757034
--------------------------------------------------
test_f1 : 0.840312257066425
train_f1: 0.8530620312549336
**************************************************


## Saving important objects For Deploying

In [214]:
jb.dump(Model,"Loans_Model_Final.pkl")

['Loans_Model_Final.pkl']

In [215]:
jb.dump(X.columns,"Loans_Columns_Final.pkl")

['Loans_Columns_Final.pkl']

In [216]:
X.describe()

,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,min_installment,ratio_of_income,diff_all
count,599.000000,614.000000,614.000000,592.000000,600.00000,564.000000,578.000000,578.000000,578.000000
mean,0.762938,5403.459283,1621.245798,146.412162,342.00000,0.842199,384.239880,10.120997,6691.381089
std,1.015216,6109.041673,2926.248369,85.587325,65.12041,0.364878,420.366948,20.051338,6455.041863
min,0.000000,150.000000,0.000000,9.000000,12.00000,0.000000,20.000000,0.670253,36.555556
25%,0.000000,2877.500000,0.000000,100.000000,360.00000,1.000000,228.888889,5.484486,3863.527778
50%,0.000000,3812.500000,1188.500000,128.000000,360.00000,1.000000,292.222222,7.074611,5046.388889
75%,2.000000,5795.000000,2297.250000,168.000000,360.00000,1.000000,411.111111,10.023864,7052.222222
max,3.000000,81000.000000,41667.000000,700.000000,480.00000,1.000000,7400.000000,373.931624,80200.000000


In [107]:
X.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area',
       'min_installment', 'ratio_of_income', 'diff_all'],
      dtype='object')

<a href="https://loan-eligibility-prediction-feature-engineering.streamlit.app/">App Link<a/>